In [1]:
import os
import sys
import gc
import json
import shutil
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
from collections import defaultdict
from typing import List, Dict, Optional, Tuple
from IPython.display import display

# Data handling
import numpy as np
import polars as pl
import pandas as pd

# Medical imaging
import pydicom
import cv2

# ML/DL
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast
import timm

# Transformations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Competition API
import kaggle_evaluation.rsna_inference_server

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Competition constants
ID_COL = 'SeriesInstanceUID'
LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present',
]

# FIXED: Use original model selection but keep enhancements in other areas
SELECTED_MODEL = 'ensemble'

MODEL_PATHS = {
    'tf_efficientnetv2_s': '/kaggle/input/rsna-iad-trained-models/models/tf_efficientnetv2_s_fold0_best.pth',
    'convnext_small': '/kaggle/input/rsna-iad-trained-models/models/convnext_small_fold0_best.pth',
    'swin_small_patch4_window7_224': '/kaggle/input/rsna-iad-trained-models/models/swin_small_patch4_window7_224_fold0_best.pth'
}

class InferenceConfig:
    # Model selection
    model_selection = SELECTED_MODEL
    use_ensemble = (SELECTED_MODEL == 'ensemble')
    
    # Default model settings
    image_size = 512
    num_slices = 32
    use_windowing = True
    
    # Enhanced inference settings (kept from improvements)
    batch_size = 2
    use_amp = True
    use_tta = True
    tta_transforms = 6
    
    # Enhanced processing flags
    use_modality_specific_channels = True
    use_enhanced_preprocessing = True
    
    # Ensemble weights
    ensemble_weights = {
        'tf_efficientnetv2_s': 0.4,
        'convnext_small': 0.3,
        'swin_small_patch4_window7_224': 0.3
    }

CFG = InferenceConfig()

class MultiBackboneModel(nn.Module):
    """FIXED: Compatible model that matches pretrained checkpoint structure"""
    def __init__(self, model_name, num_classes=14, pretrained=True, 
                 drop_rate=0.3, drop_path_rate=0.2):
        super().__init__()
        
        self.model_name = model_name
        
        if 'swin' in model_name:
            self.backbone = timm.create_model(
                model_name, 
                pretrained=pretrained,
                in_chans=3,
                drop_rate=drop_rate,
                drop_path_rate=drop_path_rate,
                img_size=CFG.image_size,
                num_classes=0,
                global_pool=''
            )
        else:
            self.backbone = timm.create_model(
                model_name, 
                pretrained=pretrained,
                in_chans=3,
                drop_rate=drop_rate,
                drop_path_rate=drop_path_rate,
                num_classes=0,
                global_pool=''
            )
        
        with torch.no_grad():
            dummy_input = torch.zeros(1, 3, CFG.image_size, CFG.image_size)
            features = self.backbone(dummy_input)
            
            if len(features.shape) == 4:
                num_features = features.shape[1]
                self.needs_pool = True
            elif len(features.shape) == 3:
                num_features = features.shape[-1]
                self.needs_pool = False
                self.needs_seq_pool = True
            else:
                num_features = features.shape[1]
                self.needs_pool = False
                self.needs_seq_pool = False
        
        print(f"Model {model_name}: detected {num_features} features, output shape: {features.shape}")
        
        if self.needs_pool:
            self.global_pool = nn.AdaptiveAvgPool2d(1)
        
        # FIXED: Use original metadata processing structure (2 inputs: age, sex)
        self.meta_fc = nn.Sequential(
            nn.Linear(2, 16),  # Back to original: age, sex only
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(16, 32),
            nn.ReLU()
        )
        
        # FIXED: Use original classifier structure to match checkpoints
        self.classifier = nn.Sequential(
            nn.Linear(num_features + 32, 512),  # Original size
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(drop_rate),
            nn.Linear(512, 256),  # Original size
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(drop_rate),
            nn.Linear(256, num_classes)  # Final layer
        )
        
    def forward(self, image, meta):
        # Extract image features
        img_features = self.backbone(image)
        
        # Apply appropriate pooling based on model type
        if hasattr(self, 'needs_pool') and self.needs_pool:
            img_features = self.global_pool(img_features)
            img_features = img_features.flatten(1)
        elif hasattr(self, 'needs_seq_pool') and self.needs_seq_pool:
            img_features = img_features.mean(dim=1)
        elif len(img_features.shape) == 4:
            img_features = F.adaptive_avg_pool2d(img_features, 1).flatten(1)
        elif len(img_features.shape) == 3:
            img_features = img_features.mean(dim=1)
        
        # Process metadata (back to original 2 features)
        meta_features = self.meta_fc(meta)
        
        # Combine features
        combined = torch.cat([img_features, meta_features], dim=1)
        
        # Classification
        output = self.classifier(combined)
        
        return output

def apply_dicom_windowing(img: np.ndarray, window_center: float, window_width: float) -> np.ndarray:
    """Apply DICOM windowing"""
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    img = (img - img_min) / (img_max - img_min + 1e-7)
    return (img * 255).astype(np.uint8)

def get_windowing_params(modality: str) -> Tuple[float, float]:
    """Enhanced modality-specific windowing (kept from improvements)"""
    windows = {
        'CT': (40, 80),
        'CTA': (50, 350),
        'MRA': (600, 1200),
        'MRI': (40, 80),
        'TOF': (500, 1000),
        'PC': (300, 600),
    }
    return windows.get(modality, (40, 80))

def create_enhanced_channels(volume: np.ndarray, modality: str) -> np.ndarray:
    """Enhanced channel creation (kept from improvements)"""
    if not CFG.use_modality_specific_channels:
        # Original method
        middle_slice = volume[CFG.num_slices // 2]
        mip = np.max(volume, axis=0)
        std_proj = np.std(volume, axis=0).astype(np.float32)
    else:
        # Enhanced method based on modality
        if modality in ['CTA', 'MRA']:
            # For angiographic data, use different projections
            mip = np.max(volume, axis=0)
            mean_proj = np.mean(volume, axis=0)
            center_idx = CFG.num_slices // 2
            middle_slice = volume[center_idx]
        else:
            # For CT/MRI, use original approach
            center_idx = CFG.num_slices // 2
            middle_slice = volume[center_idx]
            mip = np.max(volume, axis=0)
            mean_proj = np.mean(volume, axis=0)
    
    # Normalize projections
    if CFG.use_modality_specific_channels and modality in ['CTA', 'MRA']:
        # For angiographic data
        if mean_proj.max() > mean_proj.min():
            mean_proj = ((mean_proj - mean_proj.min()) / (mean_proj.max() - mean_proj.min()) * 255).astype(np.uint8)
        else:
            mean_proj = np.zeros_like(mean_proj, dtype=np.uint8)
        
        image = np.stack([middle_slice, mip, mean_proj], axis=-1)
    else:
        # Original approach with std projection
        std_proj = np.std(volume, axis=0).astype(np.float32)
        if std_proj.max() > std_proj.min():
            std_proj = ((std_proj - std_proj.min()) / (std_proj.max() - std_proj.min()) * 255).astype(np.uint8)
        else:
            std_proj = np.zeros_like(std_proj, dtype=np.uint8)
        
        image = np.stack([middle_slice, mip, std_proj], axis=-1)
    
    return image

def process_dicom_series(series_path: str) -> Tuple[np.ndarray, Dict]:
    """Enhanced DICOM processing (kept improvements but simplified metadata)"""
    series_path = Path(series_path)
    
    # Find all DICOM files
    all_filepaths = []
    for root, _, files in os.walk(series_path):
        for file in files:
            if file.endswith('.dcm'):
                all_filepaths.append(os.path.join(root, file))
    all_filepaths.sort()
    
    if len(all_filepaths) == 0:
        volume = np.zeros((CFG.num_slices, CFG.image_size, CFG.image_size), dtype=np.uint8)
        metadata = {'age': 50, 'sex': 0, 'modality': 'CT'}
        return volume, metadata
    
    # Process DICOM files
    slices = []
    metadata = {}
    slice_positions = []
    
    for i, filepath in enumerate(all_filepaths):
        try:
            ds = pydicom.dcmread(filepath, force=True)
            img = ds.pixel_array.astype(np.float32)
            
            # Handle multi-frame or color images
            if img.ndim == 3:
                if img.shape[-1] == 3:
                    img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2GRAY).astype(np.float32)
                else:
                    img = img[:, :, 0]
            
            # Enhanced slice position handling
            try:
                if hasattr(ds, 'SliceLocation'):
                    slice_positions.append(float(ds.SliceLocation))
                elif hasattr(ds, 'ImagePositionPatient'):
                    slice_positions.append(float(ds.ImagePositionPatient[2]))
                else:
                    slice_positions.append(i)
            except:
                slice_positions.append(i)
            
            # Extract metadata from first file
            if i == 0:
                metadata['modality'] = getattr(ds, 'Modality', 'CT')
                
                try:
                    age_str = getattr(ds, 'PatientAge', '050Y')
                    age = int(''.join(filter(str.isdigit, age_str[:3])) or '50')
                    metadata['age'] = min(age, 100)
                except:
                    metadata['age'] = 50
                
                try:
                    sex = getattr(ds, 'PatientSex', 'M')
                    metadata['sex'] = 1 if sex == 'M' else 0
                except:
                    metadata['sex'] = 0
            
            # Apply rescale if available
            if hasattr(ds, 'RescaleSlope') and hasattr(ds, 'RescaleIntercept'):
                img = img * ds.RescaleSlope + ds.RescaleIntercept
            
            # Apply enhanced windowing
            if CFG.use_windowing:
                window_center, window_width = get_windowing_params(metadata['modality'])
                img = apply_dicom_windowing(img, window_center, window_width)
            else:
                img_min, img_max = img.min(), img.max()
                if img_max > img_min:
                    img = ((img - img_min) / (img_max - img_min) * 255).astype(np.uint8)
                else:
                    img = np.zeros_like(img, dtype=np.uint8)
            
            # Resize
            img = cv2.resize(img, (CFG.image_size, CFG.image_size))
            slices.append(img)
            
        except Exception as e:
            print(f"Error processing {filepath}: {e}")
            continue
    
    # Sort slices by position if available
    if len(slice_positions) == len(slices) and len(set(slice_positions)) > 1:
        sorted_indices = np.argsort(slice_positions)
        slices = [slices[i] for i in sorted_indices]
    
    # Enhanced slice sampling
    if len(slices) == 0:
        volume = np.zeros((CFG.num_slices, CFG.image_size, CFG.image_size), dtype=np.uint8)
    else:
        volume = np.array(slices)
        if len(slices) > CFG.num_slices:
            # Use center-focused sampling
            start_idx = max(0, (len(slices) - CFG.num_slices) // 2)
            volume = volume[start_idx:start_idx + CFG.num_slices]
        elif len(slices) < CFG.num_slices:
            pad_size = CFG.num_slices - len(slices)
            volume = np.pad(volume, ((0, pad_size), (0, 0), (0, 0)), mode='edge')
    
    return volume, metadata

def get_inference_transform():
    """Standard inference transformation"""
    return A.Compose([
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

def get_tta_transforms():
    """Enhanced TTA transforms (kept from improvements)"""
    transforms = [
        A.Compose([  # Original
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        A.Compose([  # Horizontal flip
            A.HorizontalFlip(p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        A.Compose([  # Vertical flip
            A.VerticalFlip(p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        A.Compose([  # 90 degree rotation
            A.RandomRotate90(p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        A.Compose([  # Brightness/Contrast
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
        A.Compose([  # Gaussian noise
            A.GaussNoise(var_limit=(0.0, 0.01), p=1.0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    ]
    return transforms

# Global variables
MODELS = {}
TRANSFORM = None
TTA_TRANSFORMS = None

def load_single_model(model_name: str, model_path: str) -> nn.Module:
    """FIXED: Load model with proper error handling"""
    print(f"Loading {model_name} from {model_path}...")
    
    if not os.path.exists(model_path):
        print(f"Warning: Model file not found: {model_path}")
        return None
    
    try:
        # Load checkpoint
        checkpoint = torch.load(model_path, map_location=device, weights_only=False)
        
        # Extract config
        model_config = checkpoint.get('model_config', {})
        training_config = checkpoint.get('training_config', {})
        
        # Update global config if needed
        if 'image_size' in training_config:
            CFG.image_size = training_config['image_size']
        
        # Initialize model with original architecture
        model = MultiBackboneModel(
            model_name=model_name,
            num_classes=training_config.get('num_classes', 14),
            pretrained=False,
            drop_rate=0.0,
            drop_path_rate=0.0
        )
        
        # Load weights - should work now with matching architecture
        model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        model = model.to(device)
        model.eval()
        
        print(f"Successfully loaded {model_name} with best score: {checkpoint.get('best_score', 'N/A')}")
        return model
        
    except Exception as e:
        print(f"Error loading {model_name}: {e}")
        return None

def load_models():
    """Load models with better fallback handling"""
    global MODELS, TRANSFORM, TTA_TRANSFORMS
    
    print("Loading models...")
    
    if CFG.use_ensemble:
        # Load all available models for ensemble
        loaded_models = {}
        for model_name, model_path in MODEL_PATHS.items():
            model = load_single_model(model_name, model_path)
            if model is not None:
                loaded_models[model_name] = model
        
        MODELS = loaded_models
        
        # Adjust ensemble weights for available models
        if len(MODELS) > 0:
            available_models = list(MODELS.keys())
            total_weight = sum(CFG.ensemble_weights.get(name, 1.0) for name in available_models)
            CFG.ensemble_weights = {name: CFG.ensemble_weights.get(name, 1.0) / total_weight 
                                   for name in available_models}
            print(f"Adjusted ensemble weights: {CFG.ensemble_weights}")
    else:
        # Load single selected model
        if CFG.model_selection in MODEL_PATHS:
            model_path = MODEL_PATHS[CFG.model_selection]
            model = load_single_model(CFG.model_selection, model_path)
            if model is not None:
                MODELS[CFG.model_selection] = model
    
    # Initialize transforms
    TRANSFORM = get_inference_transform()
    if CFG.use_tta:
        TTA_TRANSFORMS = get_tta_transforms()
    
    print(f"Models loaded: {list(MODELS.keys())}")
    
    if len(MODELS) == 0:
        print("Warning: No models could be loaded! Will use fallback predictions.")
        return
    
    # Warm up models
    print("Warming up models...")
    dummy_image = torch.randn(1, 3, CFG.image_size, CFG.image_size).to(device)
    dummy_meta = torch.randn(1, 2).to(device)  # Back to 2 features
    
    with torch.no_grad():
        for model in MODELS.values():
            _ = model(dummy_image, dummy_meta)
    
    print("Ready for inference!")

def predict_single_model(model: nn.Module, image: np.ndarray, meta_tensor: torch.Tensor) -> np.ndarray:
    """Make prediction with a single model"""
    predictions = []
    
    if CFG.use_tta and TTA_TRANSFORMS:
        # Test time augmentation
        for transform in TTA_TRANSFORMS[:CFG.tta_transforms]:
            aug_image = transform(image=image)['image']
            aug_image = aug_image.unsqueeze(0).to(device)
            
            with torch.no_grad():
                with autocast(enabled=CFG.use_amp):
                    output = model(aug_image, meta_tensor)
                    pred = torch.sigmoid(output)
                    predictions.append(pred.cpu().numpy())
        
        # Average TTA predictions
        return np.mean(predictions, axis=0).squeeze()
    else:
        # Single prediction
        image_tensor = TRANSFORM(image=image)['image']
        image_tensor = image_tensor.unsqueeze(0).to(device)
        
        with torch.no_grad():
            with autocast(enabled=CFG.use_amp):
                output = model(image_tensor, meta_tensor)
                return torch.sigmoid(output).cpu().numpy().squeeze()

def predict_ensemble(image: np.ndarray, meta_tensor: torch.Tensor) -> np.ndarray:
    """Make ensemble prediction"""
    all_predictions = []
    weights = []
    
    for model_name, model in MODELS.items():
        pred = predict_single_model(model, image, meta_tensor)
        all_predictions.append(pred)
        weights.append(CFG.ensemble_weights.get(model_name, 1.0))
    
    # Weighted average
    weights = np.array(weights) / np.sum(weights)
    predictions = np.array(all_predictions)
    
    return np.average(predictions, weights=weights, axis=0)

def _predict_inner(series_path: str) -> pl.DataFrame:
    """Enhanced prediction logic"""
    global MODELS
    
    # Load models if not already loaded
    if not MODELS:
        load_models()
    
    # If still no models, use fallback
    if len(MODELS) == 0:
        print("No models available, using fallback predictions")
        return predict_fallback(series_path)
    
    # Extract series ID
    series_id = os.path.basename(series_path)
    
    # Process DICOM series with enhancements
    volume, metadata = process_dicom_series(series_path)
    
    # Create enhanced multi-channel input
    image = create_enhanced_channels(volume, metadata['modality'])
    
    # Prepare metadata tensor (back to 2 features: age, sex)
    age_normalized = metadata['age'] / 100.0
    sex = metadata['sex']
    meta_tensor = torch.tensor([[age_normalized, sex]], dtype=torch.float32).to(device)
    
    # Make predictions
    if CFG.use_ensemble and len(MODELS) > 1:
        final_pred = predict_ensemble(image, meta_tensor)
    else:
        # Use single model
        model = list(MODELS.values())[0]
        final_pred = predict_single_model(model, image, meta_tensor)
    
    # Create output dataframe
    predictions_df = pl.DataFrame(
        data=[final_pred.tolist()],
        schema=LABEL_COLS,
        orient='row'
    )
    
    return predictions_df

def predict_fallback(series_path: str) -> pl.DataFrame:
    """Smart fallback predictions"""
    # More informed conservative predictions based on medical knowledge
    conservative_preds = [0.05, 0.05, 0.08, 0.08, 0.12, 0.12, 0.15, 0.08, 0.08, 0.06, 0.06, 0.18, 0.10, 0.08]
    
    predictions = pl.DataFrame(
        data=[conservative_preds],
        schema=LABEL_COLS,
        orient='row'
    )
    
    # Clean up
    shutil.rmtree('/kaggle/shared', ignore_errors=True)
    
    return predictions

def predict(series_path: str) -> pl.DataFrame:
    """Top-level prediction function with robust error handling"""
    try:
        return _predict_inner(series_path)
    except Exception as e:
        print(f"Error during prediction for {os.path.basename(series_path)}: {e}")
        print("Using fallback predictions.")
        return predict_fallback(series_path)
    finally:
        # Enhanced cleanup
        shared_dir = '/kaggle/shared'
        shutil.rmtree(shared_dir, ignore_errors=True)
        os.makedirs(shared_dir, exist_ok=True)
        
        # Memory cleanup
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        gc.collect()

# Main execution
print("Initializing compatible enhanced RSNA aneurysm detection system...")
load_models()

# Initialize the inference server
inference_server = kaggle_evaluation.rsna_inference_server.RSNAInferenceServer(predict)

# Check if running in competition environment or local session
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("Running in competition mode...")
    inference_server.serve()
else:
    print("Running in local development mode...")
    inference_server.run_local_gateway()
    
    # Display results if available
    try:
        submission_df = pl.read_parquet('/kaggle/working/submission.parquet')
        print(f"Submission shape: {submission_df.shape}")
        print("Sample predictions:")
        display(submission_df.head())
        
        # Add prediction statistics
        print("\nPrediction Statistics:")
        for i, col in enumerate(LABEL_COLS):
            mean_pred = submission_df[col].mean()
            print(f"{col}: {mean_pred:.4f}")
            
    except Exception as e:
        print(f"Could not load submission file: {e}")

print("Compatible enhanced RSNA inference system ready!")

Using device: cuda
Initializing compatible enhanced RSNA aneurysm detection system...
Loading models...
Loading tf_efficientnetv2_s from /kaggle/input/rsna-iad-trained-models/models/tf_efficientnetv2_s_fold0_best.pth...
Model tf_efficientnetv2_s: detected 1280 features, output shape: torch.Size([1, 1280, 16, 16])
Successfully loaded tf_efficientnetv2_s with best score: 0.6514184588787864
Loading convnext_small from /kaggle/input/rsna-iad-trained-models/models/convnext_small_fold0_best.pth...
Model convnext_small: detected 768 features, output shape: torch.Size([1, 768, 16, 16])
Successfully loaded convnext_small with best score: 0.5659249983062382
Loading swin_small_patch4_window7_224 from /kaggle/input/rsna-iad-trained-models/models/swin_small_patch4_window7_224_fold0_best.pth...
Model swin_small_patch4_window7_224: detected 16 features, output shape: torch.Size([1, 16, 16, 768])
Successfully loaded swin_small_patch4_window7_224 with best score: 0.5642581777046055
Adjusted ensemble we

SeriesInstanceUID,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,Anterior Communicating Artery,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1.2.826.0.1.3680043.8.498.1007…",0.031416,0.031133,0.071234,0.06178,0.058167,0.061731,0.096985,0.01927,0.022185,0.02244,0.036353,0.031592,0.042471,0.305762
"""1.2.826.0.1.3680043.8.498.1002…",0.058258,0.058687,0.132593,0.127539,0.095251,0.108313,0.162329,0.036256,0.051248,0.051649,0.070337,0.05932,0.073285,0.531934
"""1.2.826.0.1.3680043.8.498.1005…",0.045877,0.057028,0.112964,0.106055,0.101288,0.112701,0.173907,0.033905,0.040761,0.048334,0.076495,0.051947,0.068365,0.549927



Prediction Statistics:
Left Infraclinoid Internal Carotid Artery: 0.0452
Right Infraclinoid Internal Carotid Artery: 0.0489
Left Supraclinoid Internal Carotid Artery: 0.1056
Right Supraclinoid Internal Carotid Artery: 0.0985
Left Middle Cerebral Artery: 0.0849
Right Middle Cerebral Artery: 0.0942
Anterior Communicating Artery: 0.1444
Left Anterior Cerebral Artery: 0.0298
Right Anterior Cerebral Artery: 0.0381
Left Posterior Communicating Artery: 0.0408
Right Posterior Communicating Artery: 0.0611
Basilar Tip: 0.0476
Other Posterior Circulation: 0.0614
Aneurysm Present: 0.4625
Compatible enhanced RSNA inference system ready!
